# Gradio Interface to Chat Description
This is the final product showcasing the CRM AI Assistant built in the Gradio environment. The current version uses DummyCRM, a test module that simulates the interface and functionality for demonstration purposes.

Our CRM AI Assistant can:

* Generate intelligent business insights and pipeline analyses using advanced prompts, providing evaluations of conversion rates, deal performance, and strategic recommendation for sales optimization (TO-DO).

* Produce detailed Matplotlib/Seaborn charts visualizing opportunity pipeline stages, revenue distribution by industry, lead conversion rates, and deal amount distributions across the sales organization.

Example queries it can handle are:
- 'Give me a summary of [account name]'
- 'Draft a follow-up email for [account name]'
- Just type any question to see what it can help with!

## Step 1: Import Packages

In [1]:
import os
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import time
import google.generativeai as genai
from dotenv import load_dotenv

/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/google/api_core/_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.6) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)


## Step 2: Setup for Gemini and API Key

In [2]:
# Load environment variables .env
load_dotenv()

# Get the Gemini API key from .env
api_key = os.getenv('GEMINI_API_KEY')

In [3]:
# Configure Gemini with API key
genai.configure(api_key=api_key)

# Debug: Print available models
print("Available models:", [m.name for m in genai.list_models()])

# Create the model
model = genai.GenerativeModel('models/gemini-pro-latest')

Available models: ['models/embedding-gecko-001', 'models/gemini-2.5-pro-preview-03-25', 'models/gemini-2.5-flash', 'models/gemini-2.5-pro-preview-05-06', 'models/gemini-2.5-pro-preview-06-05', 'models/gemini-2.5-pro', 'models/gemini-2.0-flash-exp', 'models/gemini-2.0-flash', 'models/gemini-2.0-flash-001', 'models/gemini-2.0-flash-exp-image-generation', 'models/gemini-2.0-flash-lite-001', 'models/gemini-2.0-flash-lite', 'models/gemini-2.0-flash-lite-preview-02-05', 'models/gemini-2.0-flash-lite-preview', 'models/gemini-2.0-pro-exp', 'models/gemini-2.0-pro-exp-02-05', 'models/gemini-exp-1206', 'models/gemini-2.0-flash-thinking-exp-01-21', 'models/gemini-2.0-flash-thinking-exp', 'models/gemini-2.0-flash-thinking-exp-1219', 'models/gemini-2.5-flash-preview-tts', 'models/gemini-2.5-pro-preview-tts', 'models/learnlm-2.0-flash-experimental', 'models/gemma-3-1b-it', 'models/gemma-3-4b-it', 'models/gemma-3-12b-it', 'models/gemma-3-27b-it', 'models/gemma-3n-e4b-it', 'models/gemma-3n-e2b-it', 'mo

## Step 3: Load Data

In [4]:
# Load clean data
base_path = "data_directory/clean_data"

# Read CSV files
accounts = pd.read_csv(os.path.join(base_path, "Accounts.csv"))
pipeline = pd.read_csv(os.path.join(base_path, "Pipeline.csv"))
teams = pd.read_csv(os.path.join(base_path, "Teams.csv"))
products = pd.read_csv(os.path.join(base_path, "Products.csv"))

## Step 4: Import Agent Classes

In [5]:
from agents import EmailDraftingAgent, AccountSummaryAgent, InsightGenerationAgent

### CRM and Search Modules

In [6]:
class CRM:
    """
    CRM class that routes queries to appropriate agents.
    """
    
    def __init__(self, accounts_df, pipeline_df, teams_df, products_df):
        self.accounts = accounts_df
        self.pipeline = pipeline_df
        self.teams = teams_df
        self.products = products_df
        
        # Initialize agents
        print("Initializing AI agents...")
        self.account_agent = AccountSummaryAgent()
        self.email_agent = EmailDraftingAgent()
        print("AI agents ready")
    
    def process_query(self, query, context=None):
        """
        Process user queries and route to appropriate agent.
        """
        query_lower = query.lower()
        
        # Check for account summary requests
        if any(word in query_lower for word in ['summary', 'summarize', 'overview', 'tell me about']):
            return self._handle_account_summary(query, context)
        
        # Check for email drafting requests
        elif any(word in query_lower for word in ['email', 'draft', 'write', 'compose', 'send']):
            return self._handle_email_request(query, context)
        
        # Default: provide helpful guidance
        else:
            return self._handle_general_query(query, context)
    
    def _handle_account_summary(self, query, context):
        """Handle account summary requests."""
        try:
            # Try to get account from context
            account_name = None
            
            # First check if 'account' key exists in context
            if context and 'account' in context:
                account_name = context['account']
            
            # If not found, try to extract from query
            if not account_name:
                for acc in self.accounts['account'].unique():
                    if acc.lower() in query.lower():
                        account_name = acc
                        break
            
            if not account_name:
                return "Please specify an account name or select one from the filter.\n\nExample: 'Give me a summary of [account name]'"
            
            print(f"Looking up account: {account_name}")
            
            # Get account data
            account_row = self.accounts[self.accounts['account'] == account_name]
            
            if account_row.empty:
                return f"Account '{account_name}' not found. Please check the spelling or select from the dropdown."
            
            account_info = account_row.iloc[0]
            
            # Prepare account data with safe column access
            account_data = {
                'account_name': str(account_info.get('account', account_name)),
                'sector': str(account_info.get('sector', 'Unknown')),
                'revenue': float(account_info.get('revenue', 0)) if pd.notna(account_info.get('revenue')) else 0,
                'employees': int(account_info.get('employees', 0)) if pd.notna(account_info.get('employees')) else 0,
                'location': str(account_info.get('office_location', 'Unknown'))
            }
            
            print(f"Account data prepared: {account_data['account_name']}")
            
            # Get opportunities for this account
            account_opps = self.pipeline[self.pipeline['account'] == account_name]
            if not account_opps.empty:
                won_deals = account_opps[account_opps['deal_stage'].str.lower() == 'won']
                account_data['total_opportunities'] = len(account_opps)
                account_data['won_deals'] = len(won_deals)
                account_data['total_revenue'] = float(won_deals['close_value'].sum())
                print(f"💼 Found {len(account_opps)} opportunities")
            
            # Calculate simple health score
            health_score = 0.75  # Default
            if not account_opps.empty:
                won_count = len(won_deals)
                total_closed = len(account_opps[account_opps['deal_stage'].str.lower().isin(['won', 'lost'])])
                if total_closed > 0:
                    health_score = won_count / total_closed
            
            print(f"Health score: {health_score:.2f}")
            
            # Generate summary using agent
            print(f"Generating AI summary...")
            summary = self.account_agent.create_summary(account_data, health_score)
            
            print(f"Summary generated successfully!")
            return summary
            
        except Exception as e:
            # Show the actual error for debugging
            import traceback
            error_details = traceback.format_exc()
            print(f"ERROR:\n{error_details}")
            return f"Error generating account summary:\n\n{str(e)}\n\nPlease check the console for details."
    
    def _handle_email_request(self, query, context):
        """Handle email drafting requests."""
        try:
            # Determine email type from query
            if 'follow' in query.lower():
                email_type = 'follow_up'
            elif 'intro' in query.lower():
                email_type = 'introduction'
            elif 'proposal' in query.lower():
                email_type = 'proposal'
            elif 'check' in query.lower():
                email_type = 'check_in'
            else:
                email_type = 'follow_up'  # Default - Fixed: underscore
            
            print(f"Email type: {email_type}")
            
            # Get account from context
            account_name = context.get('account') if context else None
            
            if not account_name:
                return "Please select an account from the filter to draft an email."
            
            print(f"🔍 Drafting email for: {account_name}")
            
            # Get account data
            account_row = self.accounts[self.accounts['account'] == account_name]
            
            if account_row.empty:
                return f"Account '{account_name}' not found."
            
            account_info = account_row.iloc[0]
            
            account_data = {
                'account_name': str(account_info.get('account', account_name)),
                'sector': str(account_info.get('sector', 'Unknown')),
                'revenue': float(account_info.get('revenue', 0)) if pd.notna(account_info.get('revenue')) else 0,
                'employees': int(account_info.get('employees', 0)) if pd.notna(account_info.get('employees')) else 0
            }
            
            # Get opportunity data if exists
            opportunity_data = None
            account_opps = self.pipeline[self.pipeline['account'] == account_name]
            
            if not account_opps.empty:
                opp = account_opps.iloc[0]
                opportunity_data = {
                    'product': str(opp.get('product', 'Unknown')),
                    'deal_stage': str(opp.get('deal_stage', 'Unknown')),
                    'close_value': float(opp.get('close_value', 0)) if pd.notna(opp.get('close_value')) else 0
                }
                print(f"Opportunity found: {opportunity_data['product']}")
            
            # Determine tone
            tone = 'friendly' if 'friendly' in query.lower() else 'professional'
            
            # Generate email using agent
            print(f"Generating {email_type} email...")
            email = self.email_agent.draft_email(
                email_type=email_type,
                account_data=account_data,
                opportunity_data=opportunity_data,
                tone=tone
            )
            
            print(f"Email generated successfully!")
            return email
            
        except Exception as e:
            # Show the actual error for debugging
            import traceback
            error_details = traceback.format_exc()
            print(f"ERROR:\n{error_details}")
            return f"Error generating email:\n\n{str(e)}\n\nPlease check the console for details."
    
    def _handle_general_query(self, query, context):
        """Handle general queries with helpful guidance."""
        
        # Provide some quick stats
        response = "**CRM AI Assistant - What I Can Help With:**\n\n"
        response += "**Account Summaries**: Ask me to summarize any account\n"
        response += "   Example: 'Give me a summary of [account name]'\n\n"
        response += "**Email Drafting**: I can draft sales emails for you\n"
        response += "   Example: 'Draft a follow-up email for [account name]'\n\n"
        response += "**Quick Stats**:\n"
        response += f"   • Total Accounts: {len(self.accounts)}\n"
        response += f"   • Total Opportunities: {len(self.pipeline)}\n"
        response += f"   • Won Deals: {len(self.pipeline[self.pipeline['deal_stage'].str.lower() == 'won'])}\n\n"
        response += "**Tip**: Select an account from the filter and ask me about it!"
        
        return response

In [7]:
# Create the CRM instance
crm = CRM(
    accounts_df=accounts,
    pipeline_df=pipeline,
    teams_df=teams,
    products_df=products
)

Initializing AI agents...
AI agents ready


### Data Visualization & Analytics

In [8]:
# Task 3c: Data Visualization & Analytics (Starter)
def generate_chart(account=None, agent=None, timeframe=None):
    fig, ax = plt.subplots(figsize=(10, 11))
    df = pipeline.copy()

    if account:
        df = df[df["account"] == account]
    if agent:
        df = df[df["sales_agent"] == agent]
    if timeframe:
        df = df[df["engage_date"] >= timeframe]

    summary = df.groupby("deal_stage")["close_value"].sum()
    summary.plot(kind="bar", ax=ax)
    ax.set_title("Deal Value by Stage")
    ax.set_ylabel("Close Value ($)")
    ax.set_xlabel("Deal Stage")

    return fig

### Chatbot Implementation

In [9]:
# CRM chatbot function
def crm_chatbot(user_query, account_filter, agent_filter, timeframe_filter):
    context = {}
    if account_filter and account_filter != "None":
        row = accounts[accounts["account"] == account_filter].iloc[0].to_dict()
        context.update(row)
    if agent_filter and agent_filter != "None":
        context["sales_agent"] = agent_filter
    if timeframe_filter:
        context["timeframe"] = timeframe_filter

    llm_response = crm.process_query(user_query, context=context)
    chart = generate_chart(
        account_filter if account_filter != "None" else None,
        agent_filter if agent_filter != "None" else None,
        timeframe_filter
    )
    return llm_response, chart

In [10]:
# wrapper to handle "None" dropdowns
account_choices = ["None"] + sorted(accounts["account"].unique())
agent_choices = ["None"] + sorted(pipeline["sales_agent"].unique())

def crm_chatbot_wrapper(user_query, account_filter, agent_filter, timeframe_filter):
    if account_filter == "None":
        account_filter = None
    if agent_filter == "None":
        agent_filter = None
    return crm_chatbot(user_query, account_filter, agent_filter, timeframe_filter)

### Final Gradio Interface

In [11]:
# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("<h1 style='text-align: center; font-size: 40px;'>CRM AI Assistant</h1>")
    gr.Markdown("<h2 style='text-align: center; font-size: 20px;'>Salesforce Team2B</h2>")
    gr.Markdown("<p style='text-align: center; font-size: 15px;'>Our CRM AI Assistant is your go-to buddy for valuable insights. It’ll help you get any answer you need!</p>")
    gr.Markdown("<p style='text-align: center; font-size: 16px; font-weight: bold;'>Ask Questions. Get Insights. See Charts.</p>")

    with gr.Row():
        user_query = gr.Textbox(label="Your Question")
    
    with gr.Row():
        account_filter = gr.Dropdown(
            choices=account_choices,
            label="Filter by Account (optional)",
            value="None"
        )
        agent_filter = gr.Dropdown(
            choices=agent_choices,
            label="Filter by Sales Agent (optional)",
            value="None"
        )
        timeframe_filter = gr.Textbox(
            label="Filter by Date (YYYY-MM-DD)",
            placeholder="2025-01-01"
        )

    with gr.Row():
        submit_btn = gr.Button("Generate Insight", elem_id="submit-btn")

    gr.HTML("""
    <style>
    #submit-btn {
        background-color: #1E90FF; /* Dodger Blue */
        color: white;
        font-weight: bold;
        border-radius: 8px;
        padding: 10px 20px;
        font-size: 16px;
    }
    #submit-btn:hover {
        background-color: #104E8B; /* Darker blue on hover */
    }
    </style>
    """)

    with gr.Row():
        llm_output = gr.Textbox(label="Insight", lines=10)
    
    chart_output = gr.Plot(label="Data Visualization")

    submit_btn.click(
        crm_chatbot_wrapper,
        inputs=[user_query, account_filter, agent_filter, timeframe_filter],
        outputs=[llm_output, chart_output]
    )

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Looking up account: bioholding
Account data prepared: bioholding
💼 Found 94 opportunities
Health score: 0.59
Generating AI summary...
Summary generated successfully!
Looking up account: bioholding
Account data prepared: bioholding
💼 Found 94 opportunities
Health score: 0.59
Generating AI summary...
Summary generated successfully!
Email type: follow_up
🔍 Drafting email for: condax
Opportunity found: gtx_basic
Generating follow_up email...
Email generated successfully!
Email type: follow_up
🔍 Drafting email for: finjob
Opportunity found: mg_advanced
Generating follow_up email...
Email generated successfully!
Email type: follow_up
Email type: follow_up
🔍 Drafting email for: domzoom
Opportunity found: mg_special
Generating follow_up email...
Email generated successfully!


Traceback (most recent call last):
  File "/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/gradio/blocks.py", line 1935, in process_api
    result = await self.call_function(
  File "/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/gradio/blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/anyio/_backends/_asyncio.py", line 2485, in ru